## Conectando ao MongoDB usando Python

In [3]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
warnings.filterwarnings('ignore')

In [4]:
mongoclient = MongoClient('localhost', 27017)
db = mongoclient.thedatasocietydb
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'thedatasocietydb')

## Insert

* Não há regras de validação rígidas: qualquer tipo de documento pode ser armazenado no banco de dados. 

* Na prática, somente objetos semelhantes são armazenados na base de dados, porém se um dia aparecer um novo atributo, poderemos inclui-lo apenas nos documentos aonde o mesmo é necessário, ao contrário do modelo relacional, onde uma nova coluna seria necessária e seria aplicada a todos os registros.

* Desvantagem: 

    * Cuidado para que as bases de dados não virem uma grande confusão (vão virar, aceite desde o início)
    
    
* Vantagens:

    * Temos uma abordagem n-dimensional para representar objetos (bem… o mundo muda)
    
    * Facilidade de alteração da estrutura de dados (refatoração fria)



In [8]:
# Insere um documento na coleção autores

db.autores.insert(
  {
    "idade": 25,
    "autor": "Joao"
  }
)

ObjectId('5f2b71998ee902858326e593')

In [9]:
#Insere outro documento com estrutura diferente em autores

db.autores.insert(
  {
    "autor": "Joaquim",
    "idade": 30,
    "cidade": "Campinas"
  }
)

ObjectId('5f2b71998ee902858326e594')

In [10]:
# Insere outro documento com estrutura diferente em autores

db.autores.insert(
  {
      "nome": "Maria",
    "poemaPreferido": "Desejo – Victor Hugo"
  }
)

ObjectId('5f2b719e8ee902858326e595')

In [11]:
# Insere documento com estrutura diferente que possui outro documento (publicacao) embutido (embedded)

db.autores.insert(
  {
    "autor": "Horacio",
    "publicacao": {    "tipo": "livro",
                "titulo": "Vida dos Dinossauros",
                "ano": 2010
               }
  }
)

ObjectId('5f2b719f8ee902858326e596')

## Find

<img src="../../resources/images/find.png" width="50%">

Estrutura:

```javascript
db.collection.find({query},{projection});
```




In [21]:
# Lista todos os documentos da coleção autores

! mongo --quiet --eval 'db.autores.find({});' thedatasocietydb


# result = db.autores.find({})
# for document in result: 
#     pprint(document)
#     print()

{ "_id" : ObjectId("5f2b71998ee902858326e593"), "idade" : 25, "autor" : "Joao" }
{ "_id" : ObjectId("5f2b71998ee902858326e594"), "autor" : "Joaquim", "idade" : 30, "cidade" : "Campinas" }
{ "_id" : ObjectId("5f2b719e8ee902858326e595"), "nome" : "Maria", "poemaPreferido" : "Desejo – Victor Hugo" }
{ "_id" : ObjectId("5f2b719f8ee902858326e596"), "autor" : "Horacio", "publicacao" : { "tipo" : "livro", "titulo" : "Vida dos Dinossauros", "ano" : 2010 } }


In [22]:
! mongo --quiet --eval 'db.autores.find({}).pretty();' thedatasocietydb

{
	"_id" : ObjectId("5f2b71998ee902858326e593"),
	"idade" : 25,
	"autor" : "Joao"
}
{
	"_id" : ObjectId("5f2b71998ee902858326e594"),
	"autor" : "Joaquim",
	"idade" : 30,
	"cidade" : "Campinas"
}
{
	"_id" : ObjectId("5f2b719e8ee902858326e595"),
	"nome" : "Maria",
	"poemaPreferido" : "Desejo – Victor Hugo"
}
{
	"_id" : ObjectId("5f2b719f8ee902858326e596"),
	"autor" : "Horacio",
	"publicacao" : {
		"tipo" : "livro",
		"titulo" : "Vida dos Dinossauros",
		"ano" : 2010
	}
}


<img src="../../resources/images/oid.png" width="50%">

<img src="../../resources/images/oid2.png" width="50%">

### Condição de igualdade


In [17]:
# Lista os documentos de autores com nome = Maria

result = db.autores.find({"nome": "Maria"})

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5f2b719e8ee902858326e595'),
    'nome': 'Maria',
    'poemaPreferido': 'Desejo – Victor Hugo',
}


In [18]:
# Lista poemaPreferido de documentos de autores com nome = Maria

result = db.autores.find(
                         {"nome": "Maria"}, 
                         {
                          "_id": 0, 
                          "poemaPreferido": 1
                         }
                    )

for document in result: 
    pprint(document)

{'poemaPreferido': 'Desejo – Victor Hugo'}


In [ ]:
# Lista documentos de autores com tipo da publicacao = livro

result = db.autores.find({"publicacao.tipo": "livro"})

for document in result: 
    pprint(document)

In [ ]:
# Documentos de autores com tipo = livro e ano entre 2000 e 2015

result = db.autores.find(
                          {
                            "publicacao.tipo": "livro", 
                            "publicacao.ano": {"$gt": 1999, "$lt": 2016}
                          }
                        )
for document in result: 
    pprint(document)

## Count

In [14]:
db.autores.count_documents({})

4

In [20]:
db.autores.count_documents({"autor":"Horacio"})

1

## Update

In [ ]:
# Atualiza cidade de autor = Joaquim

db.autores.update(
                   { "autor" : "Joaquim" },
                   { "$set": { 
                               "cidade": "Campinas-SP" 
                             } 
                   }
)


In [ ]:
result = db.autores.find({ "autor" : "Joaquim" })

for document in result: 
    pprint(document)

In [ ]:
result = db.autores.find({ "nome" : "Maria" })

for document in result: 
    pprint(document)

In [ ]:
# Adiciona atributo cidade ao autor de nome = Maria

db.autores.update(
    {"nome": "Maria"}, 
    {"$set": {"cidade": "Campinas"}}
)

In [ ]:
result = db.autores.find({ "nome" : "Maria" })

for document in result: 
    pprint(document)

In [ ]:
# Exclui atributo poemaPreferido ao autor de nome = Maria
db.autores.update(
    {"nome": "Maria"}, 
    {"$unset": {"poemaPreferido": 1}}
)

In [ ]:
result = db.autores.find({ "nome" : "Maria" })

for document in result: 
    pprint(document)

In [ ]:
# Sobreescreve documento inteiro cujo nome = Maria
db.autores.update(
  { "nome":  "Maria" },
  { "autor": "Maria",
    "poemaPreferido": "Desejo – Victor Hugo",
    "publicacao": { 
                    "tipo": "livro",
                    "titulo": "Antes da Extinção",
                    "ano": 2014
              }
  }
)

In [ ]:
result = db.autores.find({ "autor" : "Maria" })

for document in result: 
    pprint(document)

### Operadores Update

* **upsert**: Opcional. Se = true, cria um novo documento quando não encontra um que atende ao critério de busca. Default = false.

* **multi:** Opcional. Se = true, atualiza todos os documentos que atendem ao critério de busca. Se = false, atualiza apenas o primeiro documento. Default = false.

In [ ]:
# Atualiza todos os documentos com titulo Vida dos Dinossauros
db.autores.update(
                  { "publicacao.titulo":"Vida dos Dinossauros"},
                  { "$set": {"publicacao.ano": 2011}},
                  False, #upsert
                  True   #multi
)

## Remove

In [ ]:
# Exclui documento cujo autor = Maria

db.autores.remove({"autor": "Maria"})

In [ ]:
# Exclui todos os documentos da coleção

db.autores.remove({})